In [1]:
cd /Users/alessandropreviero/Desktop/vaccine-allocation/src

/Users/alessandropreviero/Desktop/vaccine-allocation/src


## Load saved solution

In [42]:
import pandas as pd
import datetime as dt
import numpy as np
import pickle
from pipeline.data_loading import load_and_clean_delphi_predictions
from models.prescriptive_delphi_model import DELPHISolution
import plotly.graph_objects as go

In [34]:
solution_dir = '/Users/alessandropreviero/Desktop/vaccine-allocation/data/outputs/optimized-solution-0.pickle'
delphi_dir = '/Users/alessandropreviero/Desktop/vaccine-allocation/data/inputs/delphi-predictions.csv'

In [35]:
with open(solution_dir, 'rb') as f:
    solution = pickle.load(f)

In [36]:
optimal_locations = solution.locations
infectious = solution.infectious
hospitalized_recovering = solution.hospitalized_recovering
quarantined_recovering = solution.quarantined_recovering
deceased = solution.deceased

## Plot DELPHI predicted deaths

In [60]:
delphi_pred_df = load_and_clean_delphi_predictions(delphi_dir)
vect = delphi_pred_df[(delphi_pred_df.date >= dt.datetime(2021, 1, 30)) &
               (delphi_pred_df.date <= dt.datetime(2021, 4, 30))
              ][['date','total_detected_deaths', 'state']]

In [59]:
import plotly.express as px
px.scatter(vect, x='date', y='total_detected_deaths', color='state')

## Plot active cases per state across time

In [8]:
active_cases = infectious + hospitalized_recovering + quarantined_recovering
active_cases = active_cases.sum(axis=1).T

In [9]:
states = pd.read_csv('/Users/alessandropreviero/Desktop/vaccine-allocation/data/inputs/county_pop_clean.csv')
states_names = states['state'].unique()

In [10]:
active_cases_df = pd.DataFrame(active_cases, columns=states_names)

In [19]:
active_plot = go.Figure()

for i in range(active_cases_df.shape[1]):
    active_plot.add_trace(
        go.Scatter(
            x=np.arange(91), 
            y=active_cases_df.iloc[:, i],
            mode='lines',
            name=active_cases_df.columns[i]
        )
    )
active_plot.update_layout(
    title="Active cases towards recovery by state"
)
active_plot.show()

## Plot cumulative deaths per state

In [30]:
deceased = solution.deceased.sum(axis=1).T
deceased_df = pd.DataFrame(deceased, columns=states_names)

In [32]:
deaths_plot = go.Figure()

for i in range(deceased_df.shape[1]):
    deaths_plot.add_trace(
        go.Scatter(
            x=np.arange(91), 
            y=deceased_df.iloc[:, i],
            mode='lines',
            name=deceased_df.columns[i]
        )
    )
deaths_plot.update_layout(
    title="Cumulative deaths towards recovery by state"
)
deaths_plot.show()

# OLD STUFF

In [62]:
active_cases_flat = active_cases.flatten('F')

In [63]:
states = pd.read_csv('/Users/alessandropreviero/Desktop/vaccine-allocation/data/inputs/county_pop_clean.csv')
states_names = states['state'].unique()
repeated_arr = np.tile(states_names, 91)

In [64]:
long_df = pd.DataFrame({'states': repeated_arr, 'active_cases':active_cases_flat})
long_df['day'] = np.repeat(np.arange(91), 51)

In [65]:
locations_df = pd.DataFrame({'state': states_names, 'num_locations': optimal_locations})

In [66]:
long_df.to_csv('optimal_active_cases_map.csv')
locations_df.to_csv('optimal_num_locations_states.csv')

In [67]:
deaths = solution.deceased[:,:,-1].sum(axis=1)
locations_df['deaths'] = deaths

In [69]:
locations_df.sort_values('state', ascending=True)

,state,num_locations,deaths
0,AK,1.0,658.980806
1,AL,1.0,90.488332
2,AR,4.0,2062.081740
3,AZ,1.0,651.689202
4,CA,1.0,2536.146179
5,CO,2.0,602.858111
6,CT,2.0,922.552851
7,DC,1.0,122.359611
8,DE,1.0,81.046338
9,FL,14.0,19145.998170
